In [12]:
from general_utils import *
from own_utils import *

In [13]:
# Check if polynom is well factorized
mults = [
    [1, 1],
    [1, 1],
    [1, 0, 1, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 0, 1],
]

ans = mults[0]
for i in range(1, 6): 
    ans = multiply_gf2(ans, mults[i])
    
print(ans)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [14]:
# Create systematic generator matrix G = [I | P]
# where I is k×k identity matrix and P is k×(n-k) parity matrix
I = np.eye(k)

# Create parity matrix P based on the generating polynomial g(x) = x^4 + x^3 + x^2 + 1
# The polynomial coefficients are [1, 1, 1, 0, 1] (from highest to lowest degree)
g = np.array([1, 1, 1, 0, 1])

# Create parity matrix P
P = np.zeros((k, n-k), dtype=int)
for i in range(k):
    dividend = np.zeros(n, dtype=int)
    dividend[i] = 1  # x^(n-1-i)
    
    remainder = polynomial_division(dividend, g)
    P[i] = remainder


print("Parity matrix P:")
print(P)


Parity matrix P:
[[1 1 1 0]
 [0 1 1 1]
 [1 1 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [1 1 1 0]
 [0 1 1 1]
 [1 1 0 1]]


In [15]:
# Find minimum rows that sum to zero vector
min_rows = find_min_rows_sum_to_zero(P)
print(f"dmin = {len(min_rows)}")
print(f"One possible example: {min_rows}")

dmin = 2
One possible example: (0, 7)


In [16]:
# Proposed parity matrix
P = np.matrix([[1, 1, 0, 0],
     [0, 1, 1, 0],
     [0, 0, 1, 1],
     [1, 0, 0, 1],
     [1, 1, 1, 0],
     [0, 1, 1, 1],
     [1, 0, 1, 1],
     [1, 1, 0, 1],
     [1, 1, 1, 1],
     [1, 0, 1, 0]], dtype=np.int64)
G = np.hstack((np.eye(k), P))
H = np.hstack((P.T, np.eye(n-k)))

print("\nGenerator matrix G:")
print(G)


Generator matrix G:
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0.]]


In [17]:
# Find minimum rows that sum to zero vector
min_rows = find_min_rows_sum_to_zero(P)
print(f"dmin = {len(min_rows)}")
print(f"One possible example: {min_rows}")

dmin = 3
One possible example: (0, 1, 9)


In [18]:
# Check error detection for some specific pattern
error = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

syndrome = np.dot(error, H.T) % 2
print("syndrome", syndrome)
if not np.any(syndrome):  # If syndrome is zero
    print(f"Error! Single-bit error pattern {error} not detected")

syndrome [[1. 0. 1. 0.]]


In [19]:
# Test single-bit errors (t_correct = 1)
print("\nError Correction Capability (t_correct = 1):")
single_errors = generate_error_patterns(1, n)
print("single", len(single_errors))

for error in single_errors:
    # Calculate syndrome: s = e * H^T
    syndrome = np.dot(error, H.T) % 2
    if not np.any(syndrome):  # If syndrome is zero
        print(f"Error! Single-bit error pattern {error} not detected")
print("All single-bit errors are correctable ✓")

# Test double-bit errors (t_detect = 2)
print("\nError Detection Capability (t_detect = 2):")
double_errors = generate_error_patterns(2, n)
print("double", len(double_errors))

for error in double_errors:
    syndrome = np.dot(error, H.T) % 2
    if not np.any(syndrome):  # If syndrome is zero
        print(f"Error! Double-bit error pattern {error} not detected")
print("All double-bit errors are detectable ✓")

# Test triple-bit errors (should be some undetectable)
print("\nTesting triple-bit errors (should be some undetectable):")
triple_errors = generate_error_patterns(3, n)
print("triple", len(triple_errors))

undetectable = 0
for error in triple_errors:
    syndrome = np.dot(error, H.T) % 2
    if not np.any(syndrome):  # If syndrome is zero
        undetectable += 1
        if undetectable <= 3:
            print(f"Some example of undetectable triple-bit error pattern: {error}")
print(f"Found {undetectable} undetectable triple-bit error patterns")



Error Correction Capability (t_correct = 1):
single 14
All single-bit errors are correctable ✓

Error Detection Capability (t_detect = 2):
double 105
All double-bit errors are detectable ✓

Testing triple-bit errors (should be some undetectable):
triple 469
Some example of undetectable triple-bit error pattern: [[1 1 0 0 0 0 0 0 0 1 0 0 0 0]]
Some example of undetectable triple-bit error pattern: [[1 0 1 0 0 0 0 0 1 0 0 0 0 0]]
Some example of undetectable triple-bit error pattern: [[1 0 0 0 1 0 0 0 0 0 0 0 1 0]]
Found 28 undetectable triple-bit error patterns


In [20]:
# Test double bit errors
double_errors = generate_error_patterns(2, n)

syndrome_dict = {}
correctable = []
only_detectable = []

for error in double_errors:
    syndrome = np.dot(error, H.T) % 2
    syndrome_key = str(syndrome[0].astype(int).tolist())

    if syndrome_key in syndrome_dict:
        # If this syndrome was seen before, it's only detectable
        # Remove the previous error pattern from correctable
        prev_error = syndrome_dict[syndrome_key]
        correctable = [e for e in correctable if not np.array_equal(e, prev_error)]
        only_detectable.append((error, prev_error))
    else:
        # First time seeing this syndrome, might be correctable
        syndrome_dict[syndrome_key] = error
        correctable.append(error)

print(f"\nFound {len(correctable)} correctable double-bit errors")
print(f"Found {len(only_detectable)} only detectable double-bit errors")

print("\nExample of correctable double-bit errors:")
for i, error in enumerate(correctable[:3]):
    print(f"Error pattern {i+1}: {error}")

print("\nExample of only detectable double-bit errors:")
for i, (error1, error2) in enumerate(only_detectable[:3]):
    print(f"Pair {i+1}:")
    print(f"Error pattern 1: {error1}")
    print(f"Error pattern 2: {error2}")
    print(f"Both have syndrome: {np.dot(error1, H.T) % 2}")

print(correctable)
print(only_detectable)


Found 0 correctable double-bit errors
Found 90 only detectable double-bit errors

Example of correctable double-bit errors:

Example of only detectable double-bit errors:
Pair 1:
Error pattern 1: [[1 1 0 0 0 0 0 0 0 0 0 0 0 0]]
Error pattern 2: [[0 0 0 0 0 0 0 0 0 1 0 0 0 0]]
Both have syndrome: [[1. 0. 1. 0.]]
Pair 2:
Error pattern 1: [[1 0 1 0 0 0 0 0 0 0 0 0 0 0]]
Error pattern 2: [[0 0 0 0 0 0 0 0 1 0 0 0 0 0]]
Both have syndrome: [[1. 1. 1. 1.]]
Pair 3:
Error pattern 1: [[1 0 0 0 1 0 0 0 0 0 0 0 0 0]]
Error pattern 2: [[0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
Both have syndrome: [[0. 0. 1. 0.]]
[]
[(array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])), (array([[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])), (array([[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])), (array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]), array([[0, 0, 0, 0, 